In [2]:
import pandas as pd

# Example: load 2023 season play-by-play
url = "https://github.com/nflverse/nflfastR-data/raw/master/data/play_by_play_2024.parquet"
df = pd.read_parquet(url)

df.head()

HTTPError: HTTP Error 404: Not Found